# Technical Take-Home Demo- Using Claude for Python Documentation Generation

## Introduction

Introduction:
In this guide, we will show how to use Claude in order to understand and document code that has no docstrings. A primary motivator for this is that documentation is incredibly import for human-legible code, but it can also be used to gain a better understanding of new code or built into something bigger to generate an API.

This notebook is meant to be run in order due to a few helper functions. We will work through the following steps. 

1. Connect with Claude and open our code sample file
2. Use AST and Claude to generate analysis of code
3. Use Claude to insert docstrings
4. Use Claude to generate a readme file

When you are done, we will have generated 3 extra files:

`code_analysis.json`
`documented_code.py`
`generated_readme.md`

Useful Links before we begin


*   [Anthropic API Documentation](https://docs.anthropic.com/en/home)

*   [Claude Cookbook](https://github.com/anthropics/anthropic-cookbook)




## Install dependancies, Get API Key, Connect to Claude

If you need help with this section, follow these steps here:
[Getting started with the Claude SDK](https://github.com/anthropics/courses/blob/master/anthropic_api_fundamentals/01_getting_started.ipynb)

In the next sections we are installing our dependacies and intializing our connection to Claude.

In [54]:
# First, install dependancies for this notebook
%pip install anthropic
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\kfeze\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\kfeze\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Next steps are to import required python libraries, most notably AST and Anthopic. 

In [55]:
from dotenv import load_dotenv
from anthropic import Anthropic
import ast
import os
import json
from pathlib import Path
from typing import Dict, List, Optional, Tuple

#load environment variable
load_dotenv()

#automatically looks for an "ANTHROPIC_API_KEY" environment variable, stored in the .env file
client = Anthropic()

The below are a few helper functions. The first is an example using a prompt plus some other content. For us, we use it to generate the Readme as well as the AST analysis. The second helper function does the treewalk that generates the AST report.

In [56]:
def ask_claude_with_code(prompt: str, content: str) -> str:
    """
    Make API call to Claude.

    Args:
        prompt: The prompt to send to Claude
        content: Code or other files for Claude to analyze based on the prompt

    Returns:
        Claude's response text
    """
    response = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        max_tokens=2000,
        system=prompt,
        messages=[
            {"role": "user", "content": f"{content}"}
        ],
    )
    print(response.content[0].text)
    return(response.content[0].text)

def extract_ast_info(tree: ast.AST) -> Dict:
    """Extract structural information from AST."""
    classes = []
    functions = []
    imports = {"external": [], "internal": []}

    for node in ast.walk(tree):
        if isinstance(node, ast.ClassDef):
            methods = [n.name for n in node.body if isinstance(n, ast.FunctionDef)]
            classes.append({
                "name": node.name,
                "methods": methods,
                "line": node.lineno
            })
        elif isinstance(node, ast.FunctionDef):
            if not any(node.lineno >= cls["line"] for cls in classes):  # Not a method
                functions.append({
                    "name": node.name,
                    "args": [arg.arg for arg in node.args.args],
                    "line": node.lineno
                })
        elif isinstance(node, ast.Import):
            for alias in node.names:
                imports["external"].append(alias.name)
        elif isinstance(node, ast.ImportFrom):
            if node.module:
                imports["external"].append(node.module)

    return {
        "classes": classes,
        "functions": functions,
        "imports": imports,
        "total_lines": len(tree.body) if hasattr(tree, 'body') else 0
    }



Here we are opening our file for running AST as well as sending to Claude via a chat. Note: feel free to change what file gets opened, however, this project is only scoped with .py files

In [57]:
#open a local copy for AST + local processing
file_name="main.py" #modify this for other files to process
if not file_name.endswith(".py"):
  print("We are only focusing on Python files for the purpose of this demo")
else:
  try:
      with open(file_name, "rb") as file:
          content = file.read()
  except FileNotFoundError:
      print("The file was not found.")



## Structural Interpretation

The first section of this code uses AST, or Abstract Syntax Trees, which is an extremely usful tool for working with code. 
You can read more about it [here](https://docs.python.org/3/library/ast.html)

In simple form, it allows us to parse python code and extract information about requirements, methods, classes, and functions. We need these in order to properly understand the code and document it via Claude.

In [ ]:
#Prompt for Claude to generate analysis based on AST Treewalk
STRUCTURE_ANALYSIS_PROMPT = """
You are an expert Python developer analyzing code for documentation generation.

Analyze the following Python code and provide a structured breakdown:

{content}
Please provide:

Primary Purpose: What is the main goal of this code?
Key Components: List classes, functions, and their relationships
Dependencies: External libraries and internal imports used
Complexity Level: Rate 1-5 and explain reasoning
Design Patterns: Any notable patterns or architectural decisions
Error Handling: How errors are managed
Side Effects: Any file I/O, network calls, state changes

Format your response as structured JSON:
json{
  "primary_purpose": "...",
  "key_components": [...],
  "dependencies": {...},
  "complexity_level": {...},
  "design_patterns": [...],
  "error_handling": "...",
  "side_effects": [...]
}
"""

The next cell uses our helper functions to parse the tree with AST and send that information to Claude to generate that syntax analysis in a more readable format.

In [ ]:
# Analyze the file

# First, do AST-based analysis for concrete structure
code=content
try:
    tree = ast.parse(code)
    ast_analysis = extract_ast_info(tree)
    print(ast_analysis)
except SyntaxError:
    ast_analysis = {"error": "Invalid Python syntax"}

# Then, use Claude for semantic analysis
ai_analysis = ask_claude_with_code(STRUCTURE_ANALYSIS_PROMPT, ast_analysis)
print(ai_analysis)
try:
    semantic_analysis = json.loads(ai_analysis)
except json.JSONDecodeError:
    semantic_analysis = {"error": "Failed to parse AI response"}

# Combine AST and AI analysis and write to a file
analysis= {
    "ast_analysis": ast_analysis,
    "semantic_analysis": semantic_analysis,
    "timestamp": "2025-07-31T10:30:00Z"
        }
print(analysis)
with open("code_analysis.json", "w", encoding='utf-8') as f:
        json.dump(analysis, f, ensure_ascii=False, indent=4)

{'classes': [{'name': 'PriceTracker', 'methods': ['__init__', '_setup_db', 'add_product', 'check_prices', '_extract_price', '_send_alert'], 'line': 9}, {'name': 'NotificationSettings', 'methods': ['__init__'], 'line': 86}], 'functions': [], 'imports': {'external': ['requests', 'sqlite3', 'json', 'datetime', 'smtplib', 'email.mime.text', 're'], 'internal': []}, 'total_lines': 10}
```json
{
  "primary_purpose": "A price tracking system that monitors product prices from websites, stores them in a SQLite database, and sends email alerts when prices drop below a set threshold.",
  "key_components": [
    {
      "name": "PriceTracker",
      "purpose": "Main class that handles product tracking, price checking, and notification",
      "methods": {
        "__init__": "Initializes the tracker with notification settings and database",
        "_setup_db": "Creates SQLite database schema if it doesn't exist",
        "add_product": "Adds new products to track with URL and threshold price",
   

## Docstring Generation

The next section is focused soley on generating docstrings and adding them to our original file. For this we pass both the analysis and the code file to Claude and ask it to insert docstrings, returning with a modified version of our code which we then save as a python file.

In [60]:
#Prompt for generating Docstrings
DOCSTRING_PROMPT = """
You are a senior Python developer writing comprehensive docstrings for undocumented code samples.

For this code:
{code}
Context about the broader codebase:
{context_info}
Generate a comprehensive docstrings, inserted in the apporpriate places that includes:

Clear Description: What it does AND why it exists
Parameters: Type hints and detailed explanations
Returns: What's returned and when
Raises: Specific exceptions and conditions
Example Usage: Realistic, practical examples
Notes: Important implementation details or caveats

Use Google-style docstrings. Focus on developer intent, not just mechanics.
Example output format:
pythondef your_function():
 \"\"\"Brief one-line description.

Longer description explaining the purpose and context within
the larger system. Explain WHY this function exists.

Args:
    param1 (type): Description including edge cases and constraints.
    param2 (Optional[type]): Description with default behavior.

Returns:
    type: Description of return value and possible variations.

Raises:
    ValueError: When and why this exception occurs.
    TypeError: Specific conditions that trigger this.

Example:
    >>> # Realistic usage example
    >>> result = your_function(arg1, arg2)
    >>> print(result)
    expected_output

Note:
    Important implementation details or performance considerations.
 \"\"\"

 Solely return the contents of the file. Ensure that each class, method, and function is documented. It should not contain any symbols like ```python that are not proper python code
"""

In [ ]:
# Generate docstrings for a file and save as a new file
response = client.messages.create(
    model="claude-3-7-sonnet-20250219",
    max_tokens=2000,
    system=DOCSTRING_PROMPT,
    messages=[
        {"role": "user", "content": f"{content}"},
        {"role": "user", "content": f"{analysis}"},
    ],
)
code_with_docs=response.content[0].text
print(code_with_docs)
with open('documented_code.py', 'w') as output:
    output.write(code_with_docs)

# main.py - Poorly documented but functional
import requests
import sqlite3
import json
from datetime import datetime
import smtplib
from email.mime.text import MIMEText

class PriceTracker:
    """Price tracking system that monitors products and sends alerts when prices drop.
    
    This class provides functionality to track product prices from web pages,
    store them in a SQLite database, and send alerts when prices drop below
    a specified target price.
    
    Args:
        db_path (str, optional): Path to the SQLite database file. Defaults to "prices.db".
    
    Attributes:
        db_path (str): Path to the SQLite database file.
    """
    
    def __init__(self, db_path="prices.db"):
        """Initialize the price tracker with a database path.
        
        Args:
            db_path (str, optional): Path to the SQLite database file. Defaults to "prices.db".
        """
        self.db_path = db_path
        self._setup_db()
    
    def _setup_db(self):
        """

## Readme Generation

The final step of this tutorial is to generate a Readme for our newly docstringed file. It passes the analysis back to Claude for one last time and saves the output in markdown.

In [62]:
README_PROMPT = """
You are creating a comprehensive README.md for a Python project.

Project Analysis:
- **File Structure**: {file_structure}
- **Dependencies**: {dependencies}
- **Main Components**: {components}
- **Code Complexity**: {complexity_analysis}

Create a professional README.md that includes:

1. **Project Title & Description**: Clear, compelling overview
2. **Features**: Bullet points of key capabilities
3. **Installation**: Step-by-step setup instructions
4. **Quick Start**: Minimal working example
5. **Usage Examples**: Common use cases with code
6. **API Reference**: Key functions/classes (brief)
7. **Contributing**: How others can contribute
8. **License**: Standard license section

Make it engaging and professional. Focus on what problems this solves for users.

Use proper markdown formatting with code blocks, badges, and clear sections.
"""

In [ ]:
#Generate comprehensive README
readme = ask_claude_with_code(README_PROMPT, ast_analysis)
print(readme)
# Save the generated README with proper formatting
with open('generated_readme.md', 'w', encoding='utf-8') as output:
    output.write(readme)


# Price Tracker

[![Python](https://img.shields.io/badge/Python-3.6%2B-blue)](https://www.python.org/)
[![License](https://img.shields.io/badge/License-MIT-green.svg)](https://opensource.org/licenses/MIT)

A powerful, customizable web scraper that helps you track product prices across e-commerce websites and alerts you when prices drop to your desired level.

## Features

- **Automated Price Monitoring**: Regularly checks prices of tracked products
- **Price Drop Alerts**: Sends email notifications when prices fall below your target
- **Persistent Storage**: Stores product information and price history in SQLite database
- **Multiple Website Support**: Works with various e-commerce platforms
- **Customizable Alerts**: Configure notification thresholds and email settings

## Installation

1. Clone the repository:
   ```bash
   git clone https://github.com/yourusername/price-tracker.git
   cd price-tracker
   ```

2. Install required dependencies:
   ```bash
   pip install requests beaut

## Conclusion

That's it! This can be a useful tool for your own code or trying to understand new (or legacy) code that gets handed to you. Make sure to check out the cookbook and API for more examples.
*   [Anthropic API Documentation](https://docs.anthropic.com/en/home)

*   [Claude Cookbook](https://github.com/anthropics/anthropic-cookbook)